# Logbook Metagenomics Project Salinity Stress
## abstract
## Introduction
## Workflow / Pipeline


## Logs

**Date: 5-03-2024**  

**Goals**  
- Search for tools and methods for metagenomic project

**Results**  
I don't quite know what tools and methods are used in metagenomics projects so i will first do some surface reading about metagenomics.

I was able to find a wikipedia page giving a nice overview of all the steps involved in a metagenomics. [metagenomics](https://en.wikipedia.org/wiki/Metagenomics). For us the main interest is in the non-assembled analysis, instead of assembled analysis. This is because unreliable reads should not be assambled and we don't want homunculi genomes that we then try to find the organisms for. We want to use every read as a reference to find our potential organisms. This is called classification.

This was a good reference project [here](https://pmc.ncbi.nlm.nih.gov/articles/PMC8086013/)  

I also found nanoclass2 which can taxonomic classify using a bunch of tools, though it seems this is tool does everything for you, which might not be something for us [here](https://ndombrowski.github.io/NanoClass2/source/run_nanoclass.html)  


I found some classifiction tools that are more wide spread.  
Kraken2, Kaiju, MegaBLAST, RDP, IDTAXA, [SPINGO](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-015-0747-1)  

A big question is which classifiers should we use to guarentee that the classified organisms are accurate. Because we don't know what organisms to expect and want to do a broad as possible search, a combination of these tools would be good choice.  

Porechop for trimming adapters   
Filtlong or nanofilt for filtering reads based on quality.  
Kaiju, Kraken2, RDP and MegaBLAST for Taxonimy classification.  


**Refelction**  
There is a large variety of tools and directions a metagenommics project can go. I am having a hard time creating something concreate, Ronald is hinting, sortof, but i am only more confused on what would be the best direction for our project. I know for sure classifing will get us a good way. another potential direction might be comparitive metagenomics.

**Next Steps**  
- Testing Trimming tools  

---

**Date: 11-03-2025**  

**Goals**  
- Setting up Kaiju in environment

**Results**  
After having a chat with Martijn, we have come to learn that our data has already been preprocessed. This means that trimming and removing adapters/barcodes is not nessisary anymore, for Martijn has already done this. Some waisted time here looking into preprocessing of data.  
The next step to processing our data will be to classify them and see what organisms exists in our samples. I already collected a few tools that could do this, but Martijn noted that Kraken2 and Kaiju are our best choices. Centrifuge was the predicessor of Kraken2, and after looking a bit better RDP and SPINGO are outdated begin between 5-10 years old and not being activaly maintained.  

We also have the option to do functional annotation analaysis to see what proteins/pathways are pressent and analyse what their influence might be, but this is not relevent for awnsering our research objective. So we will omit this branch of metagenomics for future research and focus on making a good pipeline and interpreting our results. In note for future, if we can't identify a significant number of organisms, we could assemble genomes, but again this should not be nessisary.  

Kaiju can easilly be installed via conda, after installing an index needs to be build or en pre-build index needs to be downloaded. The best course of action is to use the pre-build indexes, we don't have access to any specific databases/sets so a generic one is fine. The nr database containing a subset of the NCBI BLAST-rn database is a good coverage for Archaa, bacteria and viruses. Additionally for us fungi are interessting because of known interactions between plants and fungi supporting plant growth. So the fungi database or maybe better the nr_euk database which is like nr, but also includes fungi and microbial eukaryotes.  

For now i will try and download the nr_euk and the fungi database index from [here](https://bioinformatics-centre.github.io/kaiju/downloads.html), but martijn said he will make them availible on the network as well.  


```bash
indexdir=/students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index
mkdir -p ${indexdir}/Kaiju_db_fungi ${indexdir}/kaiju_db_nr_euk
cd ${indexdir}
wget https://kaiju-idx.s3.eu-central-1.amazonaws.com/2023/kaiju_db_nr_euk_2023-05-10.tgz
wget https://kaiju-idx.s3.eu-central-1.amazonaws.com/2024/kaiju_db_fungi_2024-08-16.tgz

tar -xfv kaiju_db_nr_euk_2023-05-10.tgz -C ${indexdir}/kaiju_db_nr_euk
tar -xfv kaiju_db_fungi_2024-08-16.tgz -C ${indexdir}/kaiju_db_fungi
```


**Reflection**  
After thinking of our steps to be taken, run them by our teachers for some validation, this can save us some effort.  

**Next steps**  
- Running Kaiju on data
- Setting up Snakemake script
- Adding Kaiju command to Snakemake

---

**Date: 12-03-2025**  

**Goals**  
- Running Kaiju on test data
- Starting basic snakemake pipline with config

**Results**  
Kaiju takes 3 atleast parameters, the .fmi which is the index file, a nodes.dmp file (also from the index) and the fastq file (which can be paired end aswell). Besides that a -z can be added for multithreading, kaiju-mulit can be used to supply a list of fastq files, which is what i want to use.

I added the basis for snakemake and a snakemake config. I also discovered that kaiju-multi is not part of the conda packages. So unless i want to install i seperatly i need to make do with just normal kaiju. Snakemake should be able to handel some of the parallel processing that kaiju would have otherwise done so it should be fine. 

```bash
kaiju -t /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/nodes.dmp -f /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/kaiju_db_fungi.fmi -i /commons/Themas/Thema07/metagenomics/zoutstress/16S/zout_20_30cm_rep2/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq.gz -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kaiju/FAY08769_pass_barcode10_85e26567_1cef8301_3.out
```

I Tried running a single instance of kaiju via the cli, but i would seem kaiju is having trouble recognising the file type. I tries two different fastq files to see if i might have been a problem with just the one file, but it appears to presist. Not really sure how to fix this, ill have to ask some advice.

```bash
kaiju -t /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/nodes.dmp -f /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/kaiju_db_fungi.fmi -i /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/16s/zout_20_30cm_rep2/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kaiju/FAY08769_pass_barcode10_85e26567_1cef8301_3.out
```
running it on the test data seems to work. These files have been decompressed.

```bash
kaiju -t /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/nodes.dmp -f /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/kaiju_db_fungi.fmi -i /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq.gz -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kaiju/FAY08769_pass_barcode10_85e26567_1cef8301_3.out
```

After trying the same command on a recompressed testdata file the same error occures."Auto-detection of file type for file /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq.gz failed."

So even though in the examples of kaiju compressed fastq files are used. I seem to not be able to use them.



**Reflection**  
I definitly need to exercise more with snakemake i am having trouble writing the nessisary rules for running kaiju.

**Next steps**  
- Continue developing snakemake rules
- Run multi instance Kaiju

---


**Date: 14-03-2024**  

**Goals**
- Concatting data
- Running Kaiju on data

**Results**  
I gained some more insights on our data from Ronald. The minIOn creates a whole bunch of fastq files, but they all belong to the same sample. Meaning that all the fastq files can be concatted instead of being treated seperatly. I wrote a small bash command to handle the concatination of all the  fastq files. This will have to be incorperated into snakemake later. For now we have 8 fastq files that makes running kaiju a lot simpler.

```bash
cd /commons/Themas/Thema07/metagenomics/zoutstress/
data_dir="/students/2024-2025/Thema07/metagenomics/zoutstress/data/"
find . -maxdepth 2 -mindepth 2 -type d | parallel "mkdir -vpm 770 ${data_dir}{//}; cat {}/* > ${data_dir}{}.fastq.gz"
```

After that i also ran a bash script of kaiju to map all the samples and classifiy the reads.  

Kaiju on nr_euk dataset
```bash
name=nr_euk
srcdir=/students/2024-2025/Thema07/metagenomics/zoutstress
outdir=${srcdir}/outputs/kaiju/${name}

mkdir -p ${outdir}/WGS ${outdir}/16S
samples=(
    "WGS/zoet_20_30cm_rep1" "WGS/zoet_20_30cm_rep2" "WGS/zout_20_30cm_rep1" "WGS/zout_20_30cm_rep2" "16S/zoet_20_30cm_rep1" "16S/zoet_20_30cm_rep2" "16S/zout_20_30cm_rep1" "16S/zout_20_30cm_rep2")
for sample in "${samples[@]}";
do
    kaiju -z 30 -t ${srcdir}/kaiju_index/kaiju_db_${name}/nodes.dmp -f ${srcdir}/kaiju_index/kaiju_db_${name}/kaiju_db_${name}.fmi -i ${srcdir}/data/${sample}.fastq -o ${outdir}/${sample}.out
done
```

Kaiju on fungi dataset
```bash
name=fungi
srcdir=/students/2024-2025/Thema07/metagenomics/zoutstress
outdir=${srcdir}/outputs/kaiju/${name}

mkdir -p ${outdir}/WGS ${outdir}/16S
samples=(
    "WGS/zoet_20_30cm_rep1" "WGS/zoet_20_30cm_rep2" "WGS/zout_20_30cm_rep1" "WGS/zout_20_30cm_rep2" "16S/zoet_20_30cm_rep1" "16S/zoet_20_30cm_rep2" "16S/zout_20_30cm_rep1" "16S/zout_20_30cm_rep2")
for sample in "${samples[@]}";
do
    kaiju -z 30 -t ${srcdir}/kaiju_index/kaiju_db_${name}/nodes.dmp -f ${srcdir}/kaiju_index/kaiju_db_${name}/kaiju_db_${name}.fmi -i ${srcdir}/data/${sample}.fastq -o ${outdir}/${sample}.out
done
```
At the end of this i got a .out file for every sample. In this output file the following things are listed:
1. either C or U, indicating whether the read is classified or unclassified.
2. name of the read
3. NCBI taxon identifier of the assigned taxon

I can generate a report aswell which give the composition of reads in precentage and number, per either genus or species or other rank i specify. For now i did genus, but the consencus we made as a group is that we would work on species level.

Optionally i can add the names in the output files aswell with kaiju-addTaxonNames, but this tool is not present in the conda install. I would need to install the kaiju from git instead to use this. There does not seem to be a need to have this so i will leave it as is.

**Reflection**  
Everything worked nicely 


**Next Steps**  

 ---

**Date: 19-03-2025**  
**Goals:**  
- Generating Kaiju reports
- Converting to Krona

**Results**  

```bash
name=fungi
srcdir=/students/2024-2025/Thema07/metagenomics/zoutstress
outdir=${srcdir}/outputs/kaiju/${name}_report
mkdir -p ${outdir}/WGS ${outdir}/16S
samples=(
    "WGS/zoet_20_30cm_rep1" "WGS/zoet_20_30cm_rep2" "WGS/zout_20_30cm_rep1" "WGS/zout_20_30cm_rep2" "16S/zoet_20_30cm_rep1" "16S/zoet_20_30cm_rep2" "16S/zout_20_30cm_rep1" "16S/zout_20_30cm_rep2")
for sample in "${samples[@]}";
do
    kaijuReport -v -i ${srcdir}/outputs/kaiju/${name}/${sample}.out -o ${outdir}/${sample}.out.report -t ${srcdir}/kaiju_index/kaiju_db_${name}/nodes.dmp -n ${srcdir}/kaiju_index/kaiju_db_${name}/names.dmp -r species
done
```

- What
- Why
- How
**Reflection**  
**Next Steps**  
---

**Date: xx-xx-xxxx**  
**Goals:**  
**Results**  
- What
- Why
- How
**Reflection**  
**Next Steps**  
---